# Clustering

In [1]:
# importamos los paquetes necesarios
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# Cargamos los datos
start_base = pd.read_csv('C:\\Users\\OLX - Gonzalo\\Desktop\\tesis\\FIM_s_before.csv', sep = ',', usecols = [1,2])
not_s_base = pd.read_csv('C:\\Users\\OLX - Gonzalo\\Desktop\\tesis\\FIM_not_s.csv', sep = ',', usecols = [1,2])

In [3]:
#Unimos las columnas por itemsets iguales, solo interesan las de starters

#Primero renombro las columnasa diferentes
not_s_base = not_s_base.rename(index=str, columns = {'support':'support_not_s', 'itemsets':'itemsets'})

#ahora si haciendo left a los starters

base = pd.merge(start_base, not_s_base, how = 'left', on=['itemsets'])

In [4]:
#Calculamos el lift y nos quedamos con los de lift > 1

#completamos los na con un valor menor al min support
base.fillna(value = 0.0001)

#dividimos los dos support 
base['lift'] = base['support']/base['support_not_s']

#separamos solo los itemset que tiene lift >1
base_lift = base[base['lift']>1]


In [5]:
#transformo la columna de itemsets que esta en strings a lista de integers
itemsets_series = base_lift['itemsets'].apply(lambda x: x.strip("[]").replace("'", "").replace(" ", "").split(","))
itemsets_series = itemsets_series.apply(lambda x: [int(v_list) for v_list in x]) 
base_lift.pop('itemsets')
base_lift = pd.concat([base_lift,itemsets_series],axis = 1)


#### Funcion de distancias

https://scholarscompass.vcu.edu/cgi/viewcontent.cgi?referer=&httpsredir=1&article=1037&context=cmsc_pubs

Función de distancias si es 0 son iguales, si es 1 son completamente distintos

D = alfa * d1 + (1-alfa) * d2

d1 = 1 - [Cantidad de eventos iguales entre I1 e I2] / [Cantidad de eventos diferentes en los dos patrones ]


d2 = {Si no hay eventos iguales entre los patrones --> 1,

    Si hay al menos 1 evento igual -->
    
    SUMA[para los eventos iguales entre I1 e I2]
    
    ([Ocurrencia del evento en el data set] / 
    
    SUMA[ para los eventos diferentes entre los dos patrones ](ocurrencia de cada uno de los eventos en el dataset))}



In [6]:
#Primero hay que saber todos los eventos diferentes a considerar
v_itemsets = []
for itemset in base_lift['itemsets']:
    for event in itemset:
        if event not in v_itemsets:
            v_itemsets.append(event)        

#v_itemsets = [int(x) for x in v_itemsets]            
v_itemsets.sort()

In [7]:
#despues hay que contar la frecuencia de cada evento en el data set
# y construir una matriz EF que tenga ordenados: evento, frecuencia
'''
# cargo los dataset en tres partes y los uno
df1 = pd.read_csv('C:\\Users\\OLX - Gonzalo\\Desktop\\tesis\\event_base_1.tsv', sep = '\t')
df2 = pd.read_csv('C:\\Users\\OLX - Gonzalo\\Desktop\\tesis\\event_base_2.tsv', sep = '\t')
df3 = pd.read_csv('C:\\Users\\OLX - Gonzalo\\Desktop\\tesis\\event_base_3.tsv', sep = '\t')

df_events = pd.concat([df1, df2, df3])


basket_all = (df_events
          .groupby(['session_fake_id', 'event_id'])['session_fake_id']
          .count().unstack().reset_index().fillna(0)
          .set_index('session_fake_id'))

basket_all_events_freq = basket_all.sum(axis=0)

del(df1)
del(df2)
del(df3)
del(df_events)
del(basket_all)

basket_all_events_freq.to_csv('C:\\Users\\OLX - Gonzalo\\Desktop\\tesis\\basket_all_events_freq.csv') '''

basket_all_events_freq = pd.read_csv('C:\\Users\\OLX - Gonzalo\\Desktop\\tesis\\basket_all_events_freq.csv'
                                     , sep = ',', header = None, names = ['itemset', 'freq'])

In [8]:
#hacer una matriz Mfe (boolean_itemsets) que tenga una fila por itemset y una columna de 0 y 1 si tiene o no tiene el evento
boolean_itemsets = []
for v_list in itemsets_series:
    boolean_serie = []
    for v_int in v_itemsets:
        if v_int in v_list:
            boolean_serie.append(1)
        else:
            boolean_serie.append(0)
    boolean_itemsets.append(boolean_serie)

boolean_itemsets = np.array(boolean_itemsets)
del(boolean_serie)

In [9]:
#hacer una matriz MVClenght (common_itemsets) que sea una matriz de distancias con el # de eventos distintos entre cada par,

common_itemsets = []

for i in range(boolean_itemsets.shape[0]):
    column_common_itemsets = []
    for j in range(boolean_itemsets.shape[0]):
        v_length = np.sum(np.logical_or(boolean_itemsets[i], boolean_itemsets[j])*1)
        column_common_itemsets.append(v_length)
    common_itemsets.append(column_common_itemsets)

del(column_common_itemsets)
del(v_length)

common_itemsets = np.array(common_itemsets)

In [10]:
#hacer una matriz MVCfreq (freq_common_itemsets) que sea una matriz de distancias con el # de eventos distintos entre cada par,

events_freq = np.array(basket_all_events_freq[0:boolean_itemsets.shape[1]]['freq']).T.astype(int)
freq_common_itemsets = []

for i in range(boolean_itemsets.shape[0]):
    column_freq_itemsets = []
    for j in range(boolean_itemsets.shape[0]):
        v_freq = np.array(np.logical_or(boolean_itemsets[i], boolean_itemsets[j])*1).dot(events_freq)
        column_freq_itemsets.append(v_freq)
    freq_common_itemsets.append(column_freq_itemsets)

del(column_freq_itemsets)
del(v_freq)

freq_common_itemsets = np.array(freq_common_itemsets)

In [11]:
#Matriz de distancias d1: Mfe X Mfe[T] / [MVClength] 

d1 = 1-(boolean_itemsets.dot(boolean_itemsets.T) / common_itemsets)

In [12]:
#numerador de Matriz de distancias d2: freq de cada par comun de eventos

freq_same_itemsets = []

for i in range(boolean_itemsets.shape[0]):
    column_freq_itemsets = []
    for j in range(boolean_itemsets.shape[0]):
        v_freq = np.array(np.logical_and(boolean_itemsets[i], boolean_itemsets[j])*1).dot(events_freq)
        column_freq_itemsets.append(v_freq)
    freq_same_itemsets.append(column_freq_itemsets)

del(column_freq_itemsets)
del(v_freq)

freq_same_itemsets = np.array(freq_same_itemsets)

In [13]:
#Denominador matriz de distancias d2
denom_d2 = (freq_common_itemsets * common_itemsets)
d2 = freq_same_itemsets / denom_d2

In [14]:
#para d2 en los casos que es uno tiene que ser cero y cuando es cero tiene que ser 1
cambio_unos_d2 = (d2 == np.ones(d2.shape))*1.0
cambio_ceros_d2 = (d2 == np.zeros(d2.shape))*1.0

d2 = d2 - cambio_unos_d2 + cambio_ceros_d2

In [25]:
np.savetxt('C:\\Users\\OLX - Gonzalo\\Desktop\\tesis\\matriz_dist_d1.csv', d1, '%f', delimiter = ',')
np.savetxt('C:\\Users\\OLX - Gonzalo\\Desktop\\tesis\\matriz_dist_d2.csv', d2, '%f', delimiter = ',')
itemsets_series

In [37]:
base_lift.to_csv('C:\\Users\\OLX - Gonzalo\\Desktop\\tesis\\base_itemsets_lift_1.csv', index = False)